# prepare

In [1]:
# imports
import sys
sys.path.append("..")
import os
import argparse
import json

import torch
import pytorch_lightning as pl
import torchmetrics
import transformers

from t5_model import multitask_model
from t5_utils import Toloka_DS, Collator, SequntiaLoader


pl.utilities.seed.seed_everything(42)

/home/posokhov@ad.speechpro.com/anaconda3/envs/persona/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 't5_model'

In [ ]:
# proxy
os.environ["http_proxy"] = "http://proxy.ad.speechpro.com:3128"
os.environ["https_proxy"] = "http://proxy.ad.speechpro.com:3128"
os.environ["ftp_proxy"] = "http://proxy.ad.speechpro.com:3128"

In [ ]:
# T5 config
batch_size = 86
epochs = 15
num_warmup_steps = 1000
lr = 5e-5
# experiment config


# pretrained model

In [ ]:
# tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained("sberbank-ai/ruT5-base", truncation_side='left', padding_side='right')
special_tokens_dict = {
    "additional_special_tokens": [
        "[Model]",
        "[User]",
        "[MaleG]",
        "[FemaleG]",
        "[UnknownG]",
        "[ModelGK]",
        "[UserGK]",
        "[WorldGK]",
        "|DialogContext|:",
        "|DialogAnswer|:",
        "|DialogModelGK|:",
        "|DialogCrossEnc|:",
    ]
}
tokenizer.add_special_tokens(special_tokens_dict)

In [ ]:
# t5
t5 = transformers.T5ForConditionalGeneration.from_pretrained("sberbank-ai/ruT5-base")
t5.resize_token_embeddings(len(tokenizer))

# data

In [ ]:
# dataset
train_answer_dataset = Toloka_DS('/home/stc/persona/data/TlkPersonaChatRus/TolokaPersonaChat_gk(train).jsonl', exaples='answer', ex_per_dialog='all', context_len='all')
val_answer_dataset = Toloka_DS('/home/stc/persona/data/TlkPersonaChatRus/TolokaPersonaChat_gk(test).jsonl', exaples='answer', ex_per_dialog='all', context_len='all')

train_gk_dataset = Toloka_DS('/home/stc/persona/data/TlkPersonaChatRus/TolokaPersonaChat_gk(train).jsonl', exaples='all_gk', ex_per_dialog='all', context_len='all')
val_gk_dataset = Toloka_DS('/home/stc/persona/data/TlkPersonaChatRus/TolokaPersonaChat_gk(test).jsonl', exaples='all_gk', ex_per_dialog='all', context_len='all')

In [ ]:
# cоllator
collator = Collator(
    spectokens=special_tokens_dict["additional_special_tokens"],
    tokenizer=tokenizer,
    padding='max_length',
    qury_len=64,
    cand_len=32,
)

In [ ]:
# dataloader
train_answer_dataloader = torch.utils.data.DataLoader(
    train_answer_dataset, batch_size=batch_size, shuffle=True, collate_fn=collator.BiEncoder
)
val_answer_dataloader = torch.utils.data.DataLoader(
    val_answer_dataset, batch_size=batch_size, shuffle=True, collate_fn=collator.BiEncoder
)

train_gk_dataloader = torch.utils.data.DataLoader(
    train_gk_dataset, batch_size=batch_size, shuffle=True, collate_fn=collator.BiEncoder
)
val_gk_dataloader = torch.utils.data.DataLoader(
    val_gk_dataset, batch_size=batch_size, shuffle=True, collate_fn=collator.BiEncoder
)

In [ ]:
train_dataloader = SequntiaLoader({'answer':train_answer_dataloader, 'gk':train_gk_dataloader}, shuffle=True)
val_dataloader = SequntiaLoader({'answer':val_answer_dataloader, 'gk':val_gk_dataloader}, shuffle=True)

In [ ]:
# scheduler len
scheduler_len = len(train_dataloader) * epochs
scheduler_len

In [ ]:
train_dataloader.lens_dict, val_dataloader.lens_dict

# pl trainloop

In [ ]:
# pl model
model = single_multitask_model(
    transformer=t5,
    tokenizer=tokenizer,
    scheduler_len=scheduler_len,
    num_warmup_steps=num_warmup_steps,
    lr=lr,
)

In [ ]:
# logger
logger = pl.loggers.comet.CometLogger(
    api_key='sEJsZrYjwc0gxxUAUGQNBwTsb',
    save_dir='/home/stc/persona/logs/t5',
    project_name='T5',
    experiment_name='seq_2bienc',
)
#logger.log_hyperparams(gpt_args)

In [ ]:
# # checkpoint callback
# checkpoint_callback = pl.callbacks.ModelCheckpoint(
#      monitor='val_loss',
#      dirpath=gpt_args.save_dir,
#      filename='gpt-{epoch:02d}-{val_loss:.2f}',
#      save_top_k=1,
#      mode='min',
#  )

In [ ]:
# trainer
trainer = pl.Trainer(
    max_epochs=epochs,
    accelerator="gpu",
    devices=1,
    gradient_clip_val=1,
    logger=logger,
    num_sanity_val_steps=10,
    #callbacks=[checkpoint_callback]
)

In [ ]:
# fit 12.77
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)